Run the code cell below to begin, and select the "Hide/show all code" button to toggle code cells on/off.

In [2]:
# DATA2001 Week 2 Tutorial
# Material last updated: 28 Feb 2023
# Note: this notebook was designed with the Roboto Condensed font, which can be installed here: https://www.1001fonts.com/roboto-condensed-font.html

from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Week 2 - Data Exploration with Python

Welcome to your first tutorial for DATA2x01! We are excited to guide you through a course that should prove both challenging and relevant in a world dominated by DATA. Each week will feature a set of coding exercises, of which your tutor will demonstrate some, while others will be left as an exercise for you to complete.

The topic for this week is exploratory data analysis with Python, namely using `pandas` and `matplotlib` to perform initial data wrangling and visualisation on **Apple Music** Data from one of our 2022 tutors. Both offer detailed streaming history, but for simplicity, this tutorial focuses on the **music library** of songs itself.

Note: if you happen to be an Apple Music customer yourself, feel free to download a [copy of your own data](https://privacy.apple.com), and use this instead of the dataset we provide. Alternatively, Spotify users may still be interested in their information, and this can be downloaded [here](https://www.spotify.com/us/account/privacy), though this won’t be as useful for this tutorial (less in-depth, in JSON not CSV form, and contains other fascinating/frightening details, such as defining users based on their listening preferences e.g. "in-car listening", "fitness" or "studying or focussing"). Both also take a couple days to prepare, so you may have to return at a later date to re-crunch the numbers on your own data.

## 1. Importing Data

First, we need to import the necessary modules to run this notebook. It is **good practice** to include these at the start of your notebook so someone else running your work can see the modules required to run this on their own device.  

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1.1 Read the data

We need to import our data before we can do anything with it! Make sure you have downloaded `W2_AppleMusic.csv` from Canvas and store it in the same directory as this notebook, or adjust the filepath below if you plan on storing it elsewhere.

A first step will often involve gauging how much data we have using `.shape`, and to take a glimpse of the first few rows using `.head()`.

In [4]:
rawData = pd.read_csv('W2_AppleMusic.csv')
print("Shape of the data:", rawData.shape)
rawData.head()

FileNotFoundError: [Errno 2] No such file or directory: 'W2_AppleMusic.csv'

### 1.2 Prepare a working copy

It is once again **good practice** to make sure your raw data is immutable in case of any accidental, irreversible changes. The `.copy()` function can create a working copy of the data.

Additionally, since we are not interested in all of the 53 columns, we can take a subset using the code below. 

In [ ]:
wrkData = rawData.copy()
wrkData = wrkData[['Title', 'Artist', 'Composer', 'Album', 'Genre', 'Track Year', 'Track Number On Album', 'Track Count On Album', 'Track Duration', 'Track Play Count', 'Date Added To Library', 'Last Played Date', 'Skip Count']]

It is generally good practice to make sure your column names are of the same style. The `.rename()` function allows this to be achieved with ease.
#### Can you think of other variable naming styles?  What are some of the challenges of having spaces in variable names?

In [ ]:
wrkData.rename(columns={
    'Track Year': 'Year',
    'Track Number On Album': 'TrackNo',
    'Track Count On Album': 'AlbumTracks',
    'Track Duration': 'DurationMs',
    'Track Play Count': 'Plays',
    'Date Added To Library': 'Added',
    'Last Played Date': 'LastPlayed',
    'Skip Count': 'Skips'
}, inplace=True)
wrkData.head()

### 1.3 Initial investigation

Now that we have done some initial data wrangling, let's trial extracting smaller **subsets**, and investigate some data quality issues on the way. Notice how we can access all columns of the data and only some rows using the following code. It's also possible to extract just a few columns, by naming them in a list after the comma.

From the subset extracted below, what are some of the data quality concerns here? How could we rectify them? 

In [ ]:
wrkData.loc[[507, 870, 2736], ]

A simple step that might prove worthwhile is to drop songs where the artist field is unpopulated. We could drop all rows with blanks in any field using the basic `.dropna()` function, but this may not be wisest. Songs which haven't been played but are still in the library for example, are probably worth keeping.

We can tailor this process by only specifying the "Artist" field to drop nulls from, and confirm how the size of the dataset has changed.

In [ ]:
wrkData.dropna(subset=['Artist'], inplace=True)
wrkData.shape

## 2. Data Cleaning

Equipped with the knowledge of some key quality issues, it's important we actually try and fix some of them! This can involve a variety of different techniques depending on what data you have, and what the **goal** of your analysis is. This stage is where your duplicated data frame becomes very important, so that the raw data remains intact.

### 2.1 Removing values

If we have a look at the `Year` column, we can see that there is entries that have been inputted as 0.

In [ ]:
min(wrkData.Year)

#### Why is it problematic to leave `Year` as 0 for some rows even if we know it's a place holder?
We can change all rows where this occurs to have their `Year` replaced by `None`.

In [ ]:
wrkData.loc[wrkData['Year'] == 0, 'Year'] = None
min(wrkData.Year)

### 2.2 Date types

It is best to have a look at each column and see what `pandas` has interpreted the data type of each column to be.

In [ ]:
wrkData.dtypes

While most of these seem okay, the `Added` and `LastPlayed` columns hold Date information, but are being interpreted as text. To facilitate any calculations on these columns, we should convert to the proper data type. 

In [ ]:
wrkData['Added'] = pd.to_datetime(wrkData['Added'])

**Task: Convert the `LastPlayed` column to a date field**

Note: you will get an error when you first attempt this! Investigate why this might be, correct it, and then proceed.

In [ ]:
### TO DO

## 3. Data manipulation

### 3.1 Additional numeric columns

`DurationMs` is recorded in milliseconds, which is a level of detail beyond what's useful for our reporting. Columns can be redefined in place, such as below, where this has been converted to the nearest second. 

In [ ]:
wrkData['Duration'] = round(wrkData['DurationMs'] / 1000)
wrkData.head(1)

String operations can be undertaken as well. For example, some genres involve a split (e.g. "Hip Hop/Rap"). We may wish to naively enumerate how many extra genres are listed, and could do so with the below code (note the 4th row - index 3 - yields 1).

In [ ]:
wrkData['ExtraGenres'] = wrkData['Genre'].str.count('/')
wrkData.head()

Another column of interest could be `Composer`. Notice how for some songs there is only 1, while for others there is a few. 
#### How would we best count the number of composers for each song?
Additionally, what sort of assumptions would we be making here? These questions are important to both consider in your data pipeline, as well as to note down and document in deliverables.
<br><br>
**Task: Create a new column `ComposerCount` which counts the number of composers each song has**

In [ ]:
### TO DO

### 3.2 Additional categorical columns

We may also want to provide categorical columns which act as 'flags' for the presence of a particular value. Conditional columns can be defined using the `np.where()` function. The simple example below defines all Harry Styles songs as currently "on tour", given he is currently in Australia, then returns only songs with "Sign" in the title to confirm its success.

In [ ]:
wrkData['OnTour'] = np.where(wrkData['Artist'] == 'Harry Styles', 'Y', 'N')
wrkData.loc[wrkData['Title'].str.contains('Sign'), ]

For a more practical example, let's say we are interested in creating a flag to warn of songs that contain potentially explicit language.

We can first have a look at what `Genre`'s we have and consider which may be susceptible.

In [ ]:
wrkData.Genre.unique()

**Task: Create a new column `SFW` ("safe for work") which flags whether a song is potentially explicit**

For simplicity, define all songs with 'Rap' in the genre as unsafe to play when parents or young children are around. Feel free to extend this if desired!

Confirm your success by printing all songs with the "Black Eyed Peas" as the artist.

In [ ]:
### TO DO

## 4. Analysing data

Now that we have wrangled the data, let's actually analyse it! The form of the analysis will come down to the desired output as well as scope, but for this exercise we will do some exploratory data analysis and show some of the interesting features of the dataset.

### 4.1 Ordering

When thinking about our music data, some interesting questions may come to mind: 
- What was the last song played by the user? 
- What was the first song added to their library? 
- According to their most played songs, what is their favourite artist?

Notice how once data cleaning, though arduous, is completed, these sorts of questions can be answered with ease and little code.

**Task: Write code to answer the above questions.**

Use the [`.sort_values()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) or [`.nlargest()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nlargest.html) functions to answer the questions, and investigate the Pandas documentation (linked) to understand more about how these operations can be used.

In [ ]:
### TO DO

### 4.2 Grouping

Another technique is using one variable to aggregate over other aspects of the data. For example, we may be interested in the most prevalent artists in the dataset.

We can create a new dataframe with one row per artist rather than per song, by **"grouping by"** artist. This concept will return in later weeks of SQL, and might take a bit to get used to at first!

From there, we can instruct it to count how many titles exist for each, using the `.agg()` function, and assign that column as "Songs". From there we find the top 5, similarly to the previous question.

In [ ]:
artists = wrkData.groupby('Artist').agg(Songs = ('Title','count')).reset_index()
artists.sort_values(by=['Songs'], ascending=False).head()

**Task: Determine which genres have the greatest number of plays, and also include how many skips each genre has.**

In [ ]:
### TO DO

## 5. Visualising data

The next natural step is to visualise the data. This could be a precursor to greater statistical investigation, or stand on its own for analysis. Revising the lectures notes and/or your first year statistics will prove helpful when determining the appropriate visualisations for specific data types.

### 5.1 Histograms

Creating a histogram for songs by duration is a useful and satisfying representation of the data.

The `bins` have been set at a minimum of 0 and maximum of 1200 seconds, which excludes a few outliers (there are a couple quite long songs in the dataset). The width of each bin range has also been set to 10 seconds, which can be adjusted for more abstract or refined visuals (try changing it!).

Other aspects of the code below are purely aesthetic, such as plot titles, axis labels, colours, etc.

In [ ]:
plt.hist(wrkData['Duration'], bins=np.arange(0,1200,10), alpha=0.5, color ="#0148A4")
plt.title('Duration by Song')
plt.xlabel('Duration (seconds)')
plt.ylabel('Number of songs')
plt.grid()

Alternatively, boxplots can be used to represent the same information in a different way.

In [ ]:
plt.boxplot(wrkData.loc[wrkData['Duration'] <= 1200, 'Duration'], vert=False)
plt.title('Distribution of Duration')
plt.xlabel('Duration (seconds)')
plt.grid()

### 5.2 Bar charts with grouping

We can also return to our `.groupby()` operation for graphs. The below is quite similar, but involves a bar chart for number of songs each year.

In [ ]:
years = wrkData.groupby('Year').size().reset_index(name='Songs')
plt.bar(years['Year'], years['Songs'], alpha=0.5, color ="#0148A4", align='center')
plt.title('Number of Songs Over Time')
plt.xlabel('Year')
plt.ylabel('Number of songs')
plt.grid()

### 5.3 Free choice visual

Now that you've seen how to visualise in Python, it's time to do your own!

**Task: Consider a particular variable(s) of choice and make an interesting visualisation to explore.**

In [ ]:
### TO DO

## 6. Optional Extra Task

As an optional extension task - a practical use of this data could involve determining which genres the listener are most or least engaged with.

**Optional Task: Determine which genres have the highest/lowest skip ratio**

Only include genres with at least 10 encounters in your output, and the skip ratio = total skips divided by total encounters (the sum of plays and skips)

In [ ]:
### TO DO